# Imports

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# read csv files

In [2]:
base_dir = "/Users/zaca2954/stanislawski_lab/drift_fs/csv/processed_data/"

# load the data
pred_contrib = pd.read_csv(Path(base_dir, "pred_metagenome_contrib.tsv"), sep="\t")
pred_unstrat = pd.read_csv(Path(base_dir, "pred_metagenome_unstrat.tsv"), sep="\t")

species = pd.read_csv(Path(base_dir, "species_latent.csv"))

In [3]:
## configure the pred_unstrat dataset

# Filter columns with "BL" or taxa
pred_unstrat = pred_unstrat.filter(regex="BL|function")

# Rename the columns to only include the part before "."
pred_unstrat.columns = [col.split(".")[0] for col in pred_unstrat.columns]

# make the function row the new column names
pred_unstrat = pred_unstrat.T
pred_unstrat.columns = pred_unstrat.iloc[0]
pred_unstrat = pred_unstrat.drop("function")

In [4]:
## configure species dataset
species_df = species[['subject_id', 'diff_std_bmi_score']]

In [5]:
## configure pred_contrib dataset

# obtain only the samples from the sample column that it contains only the sampel for BL
pred_contrib = pred_contrib[pred_contrib['sample'].str.contains("BL")]

# split the sample column from "." and only obtain the first part
pred_contrib['sample'] = pred_contrib['sample'].str.split(".").str[0]

# set row names to the sample column
pred_contrib = pred_contrib.set_index("sample")

In [6]:
# merge the datasets of pred_contrib and species by index for pred_contrib and subject_id for species

# merge the datasets
merged = pred_contrib.merge(species_df, left_index=True, right_on="subject_id")

In [ ]:
# Define features (X) and target (y)
X = merged.drop(columns=["diff_std_bmi_score", 'subject_id', 'function', 'taxon'])
y = merged["diff_std_bmi_score"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Initialize the Random Forest Regressor
model = RandomForestRegressor(random_state=42, n_estimators=100)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")